## Introduction to our first task: 'Dogs vs Cats'

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
PATH = "data/dogscats/"
sz=224
batch_size=64

In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input

/home/ubuntu/anaconda3/envs/fastai-tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
train_data_dir = f'{PATH}train'
validation_data_dir = f'{PATH}valid'

In [5]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [6]:
base_model = ResNet50(weights='imagenet', include_top=False) # You have to install h5py in advance (conda install h5py)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [7]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
359/359 [==============================] - 211s 587ms/step - loss: 0.2016 - acc: 0.9491 - val_loss: 0.0580 - val_acc: 0.9854
Epoch 2/3
359/359 [==============================] - 209s 581ms/step - loss: 0.0882 - acc: 0.9695 - val_loss: 0.1123 - val_acc: 0.9773
Epoch 3/3
359/359 [==============================] - 209s 583ms/step - loss: 0.0751 - acc: 0.9732 - val_loss: 0.1248 - val_acc: 0.9747
CPU times: user 16min 4s, sys: 1min 2s, total: 17min 7s
Wall time: 10min 28s


In [9]:
split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])    

In [10]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=1, workers=3,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size) # get worse...

Epoch 1/1
359/359 [==============================] - 253s 704ms/step - loss: 0.0796 - acc: 0.9737 - val_loss: 7.8736 - val_acc: 0.5101
CPU times: user 6min 24s, sys: 25.4 s, total: 6min 49s
Wall time: 4min 13s
